# Experiment 1

**Target Task:** Cora 

**Transfer learning Source Task:** Citeseer

**Meta-learning Source Task:** Citeseer, Pubmed
***
## Installs & imports

In [ ]:
# ! pip install torch==1.5.0+cu101 torchvision==0.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

# ! pip install torch-scatter==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
# ! pip install torch-sparse==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
# ! pip install torch-cluster==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
# ! pip install torch-spline-conv==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
# ! pip install torch-geometric

# ! pip install --ignore-installed "jsonschema>=2.6.0,<3.1.0"
! pip install comet_ml --upgrade --ignore-installed

In [64]:
import numpy as np
import copy
import torch
import torch.nn.functional as F
import torch_geometric
from torch_geometric.nn import GCNConv, SAGEConv, GATConv, VGAE
from torch_geometric.data import DataLoader
from torch_geometric.datasets import Planetoid
from tqdm import tqdm
from comet_ml import Experiment

import warnings
warnings.filterwarnings("ignore")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

***
## Data & Experiment Setup

In [39]:
cora = Planetoid('./data', 'Cora')

print('Cora')
print('-----')
print('{} Nodes; {} Edges'.format(cora[0].num_nodes, cora[0].num_edges))
print('No. of features: {}'.format(cora[0].num_features))
print('No. of classes: {}'.format(cora.num_classes))

cora_data = cora[0].to(device)

Cora
-----
2708 Nodes; 10556 Edges
No. of features: 1433
No. of classes: 7


In [35]:
experiment = Experiment(project_name='experiment-1', display_summary_level=0)

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/graph-net-experiments/experiment-1/c34b75080f6a4846b6d954a6c4f43d70



***
## Training base models on Cora

### GCN

In [53]:
gcn_hyperparams = {
    'hidden_dim' : 256,
    'n_features' : cora[0].num_features,
    'n_classes' : cora.num_classes,
    'learning_rate': 0.001,
    'num_epochs': 1000,
}

In [52]:
class GCN(torch.nn.Module):
    def __init__(self, hidden_dim, n_features, n_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(n_features, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, n_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)
    
                             
gcn = GCN(hidden_dim=gcn_hyperparams['hidden_dim'],
        n_features=gcn_hyperparams['n_features'],
        n_classes=gcn_hyperparams['n_classes']
        ).to(device)
                             
optimizer = torch.optim.Adam(gcn.parameters(), lr=gcn_hyperparams['learning_rate'])

gcn

GCN(
  (conv1): GCNConv(1433, 256)
  (conv2): GCNConv(256, 7)
)

#### Training GCN on Cora

In [ ]:
gcn.train()

for epoch in tqdm(range(gcn_hyperparams['num_epochs'])):
    optimizer.zero_grad()
    out = gcn(cora_data)
    loss = F.nll_loss(out[cora_data.train_mask], cora_data.y[cora_data.train_mask])
    loss.backward()
    optimizer.step()
    
    experiment.log_metric("loss", loss, step=epoch)

In [63]:
gcn.eval()

_, pred = gcn(cora_data).max(dim=1)
correct = float (pred[cora_data.test_mask].eq(cora_data.y[cora_data.test_mask]).sum().item())
acc = correct / cora_data.test_mask.sum().item()
print('Accuracy: {:.4f}'.format(acc))

Accuracy: 0.7860


In [ ]:
experiment.end()